In [ ]:
import sklearn
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/players_20.csv')
df.describe

In [ ]:
df['player_traits'].isna().sum()

10712

In [ ]:
df['player_tags'].isna().sum()

16779

In [ ]:
new_df = df.dropna(axis=0,subset=['player_traits'])

## Loading BERT Model

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
def constructEmbeddings(sent):
  return sbert_model.encode(sent)

In [ ]:
new_df['embeddings'] = new_df['player_traits'].apply(constructEmbeddings)
# new_df = pd.read_csv('/content/drive/MyDrive/College/Sem 6/RecSys/Project2/embeddings.csv')

In [ ]:
new_df['embeddings']

0       [-0.36305606, 0.77589226, 0.55432, 0.27569258,...
1       [-0.48443338, 1.1557947, 0.91646373, 0.4993325...
2       [-0.6640025, 0.46728063, 0.8076286, 0.26393124...
3       [0.09925105, -0.5052245, 1.3611902, 0.6164345,...
4       [-0.49783152, 0.51528376, 1.1077756, 0.2626912...
                              ...                        
7561    [0.010219832, -0.21413945, 1.6238617, 0.744068...
7562    [-0.14079884, -0.10212083, 1.5475837, -0.04730...
7563    [0.0011217035, -0.4691528, 1.5869671, 0.630030...
7564    [-0.024325462, -0.46545637, 0.7181439, 0.51775...
7565    [0.23198454, 0.09484639, 0.78729177, 0.0753946...
Name: embeddings, Length: 7566, dtype: object

In [ ]:
# new_df.to_csv(r'embeddings.csv')
new_df.club.nunique()

691

In [ ]:
yes = new_df.groupby('club')['embeddings'].mean()

In [ ]:
yes

club
 SSV Jahn Regensburg     [-0.13267958, -0.02988186, 0.9670204, 0.519756...
1. FC Heidenheim 1846    [-0.06787688, -0.049798835, 1.2745366, 0.32837...
1. FC Kaiserslautern     [0.04257611, -0.23833944, 1.5059918, 0.5288948...
1. FC Köln               [-0.08155326, -0.08018833, 1.1886828, 0.289132...
1. FC Magdeburg          [-0.13525926, 0.26306897, 1.3279678, 0.4190439...
                                               ...                        
Yokohama F. Marinos      [-0.18142563, -0.0682986, 1.0145396, 0.3428907...
Zagłębie Lubin           [-0.1889395, 0.26326638, 0.8993827, 0.26150548...
Çaykur Rizespor          [-0.25040364, 0.016386658, 1.6851532, 0.391097...
Örebro SK                [0.12261691, -0.025407264, 1.2110761, 0.394760...
Östersunds FK            [-0.14924082, -0.17977697, 1.3357896, 0.472749...
Name: embeddings, Length: 691, dtype: object

## Applying cosine similarity to get list of players

In [ ]:
def applyCosine(player_vect, club_vect = yes['FC Barcelona']):
  return cosine(club_vect, player_vect)

def getRankedList(clubName):
  club_vect = yes[clubName]
  player_df = new_df[['short_name','embeddings']]
  player_df['similarity_score'] = player_df['embeddings'].apply(applyCosine)
  return player_df

out= getRankedList('FC Barcelona')
out.describe

In [ ]:
out = out.sort_values(by='similarity_score', ascending=False)
out.head(100)
out[['short_name', 'similarity_score']].to_csv('output_FCB.csv')